In [1]:
# genetic_wing_design.py
import random
import math
from typing import List, Tuple

random.seed(42)

class GeneticAlgorithm:
    def __init__(self,
                 obj_func,
                 bounds: List[Tuple[float,float]],
                 pop_size: int = 50,
                 generations: int = 200,
                 pc: float = 0.8,
                 pm: float = 0.1,
                 elitism: int = 1):
        """
        obj_func: function that takes a phenotype (list of floats) and returns fitness (higher is better)
        bounds: list of (min, max) tuples for each gene / parameter
        pop_size: number of individuals in population
        generations: number of generations to run
        pc: crossover probability
        pm: per-gene mutation probability
        elitism: number of top individuals to carry unchanged to next generation
        """
        self.obj_func = obj_func
        self.bounds = bounds
        self.nvars = len(bounds)
        self.pop_size = pop_size
        self.generations = generations
        self.pc = pc
        self.pm = pm
        self.elitism = elitism

    def _init_population(self):
        pop = []
        for _ in range(self.pop_size):
            indiv = [random.uniform(a,b) for a,b in self.bounds]
            pop.append(indiv)
        return pop

    def _evaluate(self, population):
        # returns list of fitness values (higher is better)
        fitness = []
        for indiv in population:
            fit = self.obj_func(indiv)
            fitness.append(fit)
        return fitness

    def _roulette_wheel_select(self, population, fitness):
        # fitness may be non-positive; shift if needed
        min_fit = min(fitness)
        if min_fit <= 0:
            adj = [f - min_fit + 1e-6 for f in fitness]  # make all positive
        else:
            adj = fitness[:]
        total = sum(adj)
        pick = random.uniform(0, total)
        current = 0
        for indiv, af in zip(population, adj):
            current += af
            if current >= pick:
                return indiv
        return population[-1]

    def _crossover(self, parent1, parent2):
        # BLX-alpha crossover (blend)
        alpha = 0.5
        child1 = []
        child2 = []
        for i in range(self.nvars):
            x1, x2 = parent1[i], parent2[i]
            d = abs(x1 - x2)
            low = min(x1, x2) - alpha * d
            high = max(x1, x2) + alpha * d
            a, b = self.bounds[i]
            # clamp to variable bounds
            c1 = random.uniform(low, high)
            c2 = random.uniform(low, high)
            c1 = max(a, min(b, c1))
            c2 = max(a, min(b, c2))
            child1.append(c1)
            child2.append(c2)
        return child1, child2

    def _mutate(self, individual):
        # gaussian mutation per gene
        for i in range(self.nvars):
            if random.random() < self.pm:
                a,b = self.bounds[i]
                sigma = (b - a) * 0.1
                individual[i] += random.gauss(0, sigma)
                # clamp
                individual[i] = max(a, min(b, individual[i]))
        return individual

    def run(self, verbose=False):
        population = self._init_population()
        best_solution = None
        best_fitness = -float('inf')
        history = []

        for gen in range(self.generations):
            fitness = self._evaluate(population)
            # update best
            for indiv, fit in zip(population, fitness):
                if fit > best_fitness:
                    best_fitness = fit
                    best_solution = indiv[:]
            history.append(best_fitness)
            if verbose and gen % 10 == 0:
                print(f"Gen {gen:3d} Best fitness: {best_fitness:.6f}")

            # Elitism: keep top 'elitism' individuals
            paired = list(zip(population, fitness))
            paired.sort(key=lambda x: x[1], reverse=True)
            new_pop = [p for p,_ in paired[:self.elitism]]

            # generate offspring until population filled
            while len(new_pop) < self.pop_size:
                # parent selection
                p1 = self._roulette_wheel_select(population, fitness)
                p2 = self._roulette_wheel_select(population, fitness)
                # crossover
                if random.random() < self.pc:
                    c1, c2 = self._crossover(p1, p2)
                else:
                    c1, c2 = p1[:], p2[:]
                # mutation
                c1 = self._mutate(c1)
                c2 = self._mutate(c2)
                new_pop.append(c1)
                if len(new_pop) < self.pop_size:
                    new_pop.append(c2)

            population = new_pop

        return {
            "best_solution": best_solution,
            "best_fitness": best_fitness,
            "history": history
        }

# --------------------
# Example: Synthetic "wing drag" objective (toy model)
# --------------------

TARGET = [34.2, 4.5, 12.1]

def predicted_drag(params: List[float], bounds) -> float:
    # simple squared normalized error to target -> small error => drag close to 0.02
    error = 0.0
    for p, t, (a,b) in zip(params, TARGET, bounds):
        rng = (b - a)
        if rng == 0: rng = 1.0
        error += ((p - t) / rng) ** 2
    base_drag = 0.020
    drag = base_drag + 0.015 * error
    return drag

def fitness_from_drag(params: List[float], bounds) -> float:
    drag = predicted_drag(params, bounds)
    fitness = 1.0 / (drag + 1e-6)
    return fitness

if __name__ == "__main__":
    # variable bounds: [wingspan (m), chord (m), sweep (deg)]
    bounds = [(20.0, 50.0),  # wingspan
              (1.0, 10.0),   # chord
              (0.0, 30.0)]   # sweep angle

    # wrap objective so it only takes params
    def obj(params):
        return fitness_from_drag(params, bounds)

    ga = GeneticAlgorithm(obj_func=obj,
                          bounds=bounds,
                          pop_size=50,
                          generations=200,
                          pc=0.8,
                          pm=0.1,
                          elitism=2)

    result = ga.run(verbose=True)

    best = result["best_solution"]
    best_drag = predicted_drag(best, bounds)
    best_fitness = result["best_fitness"]

    print("\n=== GA Result ===")
    print(f"Best solution (wingspan, chord, sweep): {[round(x,4) for x in best]}")
    print(f"Predicted Drag Coefficient: {best_drag:.6f}")
    print(f"Fitness (1/drag): {best_fitness:.6f}")


Gen   0 Best fitness: 48.967915
Gen  10 Best fitness: 49.923204
Gen  20 Best fitness: 49.989407
Gen  30 Best fitness: 49.990089
Gen  40 Best fitness: 49.995162
Gen  50 Best fitness: 49.995162
Gen  60 Best fitness: 49.995162
Gen  70 Best fitness: 49.995162
Gen  80 Best fitness: 49.996666
Gen  90 Best fitness: 49.996695
Gen 100 Best fitness: 49.996695
Gen 110 Best fitness: 49.996695
Gen 120 Best fitness: 49.996695
Gen 130 Best fitness: 49.996695
Gen 140 Best fitness: 49.996695
Gen 150 Best fitness: 49.997241
Gen 160 Best fitness: 49.997241
Gen 170 Best fitness: 49.997241
Gen 180 Best fitness: 49.997241
Gen 190 Best fitness: 49.997241

=== GA Result ===
Best solution (wingspan, chord, sweep): [34.1813, 4.4773, 12.0871]
Predicted Drag Coefficient: 0.020000
Fitness (1/drag): 49.997241
